In [121]:
import numpy as np
import tensorflow as tf
print(tf.__version__)

np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(linewidth=80) 

%load_ext line_profiler

2.3.0
The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [116]:
USE_PTB = False
USE_NATIVE=True

# Text to process

### Test sample text

In [117]:
#text = "The fool doth think he is wise, but the wise man knows himself to be a fool."
#text = "To be, or not to be, that is the question"
text = "I know how to build an attention in neural networks. But I don’t understand how attention layers learn the weights that pay attention to some specific embedding.I have this question because I’m tackling a NLP task using attention layer. I believe it should be very easy to learn (the most important part is to learn alignments). However, my neural networks only achieve 50% test set accuracy. And the attention matrix is weird. I don’t know how to improve my networks."

## PTB (Penn Treebank) 

In [118]:
#coding: utf-8
import sys
import os
sys.path.append('..')
try:
    import urllib.request
except ImportError:
    raise ImportError('Use Python3!')
import pickle
import numpy as np


url_base = 'https://raw.githubusercontent.com/tomsercu/lstm/master/data/'
key_file = {
    'train':'ptb.train.txt',
    'test':'ptb.test.txt',
    'valid':'ptb.valid.txt'
}
save_file = {
    'train':'ptb.train.npy',
    'test':'ptb.test.npy',
    'valid':'ptb.valid.npy'
}
vocab_file = 'ptb.vocab.pkl'

#dataset_dir = os.path.dirname(os.path.abspath(__file__))
dataset_dir = os.path.dirname(os.path.abspath("/home/oonisim/dataset/hoge"))
print(dataset_dir)

def _download(file_name):
    file_path = dataset_dir + '/' + file_name
    if os.path.exists(file_path):
        return

    print('Downloading ' + file_name + ' ... ')

    try:
        urllib.request.urlretrieve(url_base + file_name, file_path)
    except urllib.error.URLError:
        import ssl
        ssl._create_default_https_context = ssl._create_unverified_context
        urllib.request.urlretrieve(url_base + file_name, file_path)

    print('Done')


def load_text():
    data_type = 'train'
    file_name = key_file[data_type]
    file_path = dataset_dir + '/' + file_name
    vocab_path = dataset_dir + '/' + vocab_file

    if not os.path.exists(file_path):
        _download(file_name)

    text = open(file_path).read().replace('\n', '<eos>').strip()
    return(text)
    
def load_vocab():
    vocab_path = dataset_dir + '/' + vocab_file

    if os.path.exists(vocab_path):
        with open(vocab_path, 'rb') as f:
            word_to_id, id_to_word = pickle.load(f)
        return word_to_id, id_to_word

    word_to_id = {}
    id_to_word = {}
    data_type = 'train'
    file_name = key_file[data_type]
    file_path = dataset_dir + '/' + file_name

    _download(file_name)

    words = open(file_path).read().replace('\n', '<eos>').strip().split()

    for i, word in enumerate(words):
        if word not in word_to_id:
            tmp_id = len(word_to_id)
            word_to_id[word] = tmp_id
            id_to_word[tmp_id] = word

    with open(vocab_path, 'wb') as f:
        pickle.dump((word_to_id, id_to_word), f)

    return word_to_id, id_to_word


def load_data(data_type='train'):
    '''
        :param data_type: データの種類：'train' or 'test' or 'valid (val)'
        :return:
    '''
    if data_type == 'val': data_type = 'valid'
    save_path = dataset_dir + '/' + save_file[data_type]

    word_to_id, id_to_word = load_vocab()

    if os.path.exists(save_path):
        corpus = np.load(save_path)
        return corpus, word_to_id, id_to_word

    file_name = key_file[data_type]
    file_path = dataset_dir + '/' + file_name
    _download(file_name)

    words = open(file_path).read().replace('\n', '<eos>').strip().split()
    corpus = np.array([word_to_id[w] for w in words])

    np.save(save_path, corpus)
    return corpus, word_to_id, id_to_word



/home/oonisim/dataset


# Word indexing
Assign a numerical id to each word.

## Proprietary indexing

In [119]:
def preprocess(text):
    """
    Args: 
        text: A string including sentences to process.
    Returns:
        corpus: 
            A numpy array of word indices to every word in the originlal text as as they appear in the text.
            The objective of corpus is to preserve the original text but as numerical indices.
        word_to_id: A dictionary to map a word to a word index
        id_to_word: A dictionary to map a word index to a word
    """
    text = text.lower()
    text = text.replace('.', ' .')
    words = text.split(' ')

    word_to_id = {}
    id_to_word = {}
    for word in words:
        if word not in word_to_id:
            new_id = len(word_to_id)
            word_to_id[word] = new_id
            id_to_word[new_id] = word

    corpus = np.array([word_to_id[w] for w in words])

    return corpus, word_to_id, id_to_word

In [120]:
if USE_NATIVE:
    if USE_PTB:
        (corpus, word_to_id, id_to_word)= load_data('train')
    else:
        (corpus, word_to_id, id_to_word) = preprocess(text)

    print(len(corpus))
    print(len(word_to_id))
    vocab_size = max(word_to_id.values()) + 1
    print(vocab_size)
    
if not USE_PTB:
    print(word_to_id)

87
59
59
{'i': 0, 'know': 1, 'how': 2, 'to': 3, 'build': 4, 'an': 5, 'attention': 6, 'in': 7, 'neural': 8, 'networks': 9, '.': 10, 'but': 11, 'don’t': 12, 'understand': 13, 'layers': 14, 'learn': 15, 'the': 16, 'weights': 17, 'that': 18, 'pay': 19, 'some': 20, 'specific': 21, 'embedding': 22, '.i': 23, 'have': 24, 'this': 25, 'question': 26, 'because': 27, 'i’m': 28, 'tackling': 29, 'a': 30, 'nlp': 31, 'task': 32, 'using': 33, 'layer': 34, 'believe': 35, 'it': 36, 'should': 37, 'be': 38, 'very': 39, 'easy': 40, '(the': 41, 'most': 42, 'important': 43, 'part': 44, 'is': 45, 'alignments)': 46, 'however,': 47, 'my': 48, 'only': 49, 'achieve': 50, '50%': 51, 'test': 52, 'set': 53, 'accuracy': 54, 'and': 55, 'matrix': 56, 'weird': 57, 'improve': 58}


## Tensorflow Tokenizer indexing

In [69]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(oov_token="<OOV>")

USE_TENSORFLOW = (not USE_NATIVE)
if USE_TENSORFLOW:
    if USE_PTB:
        text = load_text()
    else: 
        text = text

    # Each text in "texts" is a complete document as one string, 
    # e.g "To be or not to be, that is the question."
    texts = [ text ]   

    # fit_on_texts() processes multiple documents and handles all words in all the documents.
    tokenizer.fit_on_texts(texts)
    word_to_id = tokenizer.word_index

    # texts_to_sequences() ruturns word index sequence of each document in "texts".
    sequences = (tokenizer.texts_to_sequences(texts))

    # corpus is word index sequence for a single document loaded by load_text().
    corpus = sequences[0]

    print(len(sequences))
    print(len(corpus))
    print(len(word_to_id))
    
    # Index of tokenizer.word_index starts at 1, NOT 0.
    # e.g. {'<OOV>': 1, 'the': 2, 'fool': 3, 'wise': 4, 'doth': 5, ...}
    vocab_size = max(word_to_id.values()) + 1
    print(vocab_size)

# Co-occurrence vector

In [122]:
def create_co_matrix(corpus, vocab_size, window_size=1):
    '''Generate co-occurreance matrix for the corpus.

    :param corpus:
    :param vocab_size:The number of unique words in the corpus. 
    :param window_size: 
        The number of words either left or right of the word to count co-occurreances, which is (context_ize / 2)
    :return: co-occurrence matrix
    '''
    corpus_size = len(corpus)
    co_matrix = np.zeros((vocab_size, vocab_size), dtype=np.int32)

    for idx, word_id in enumerate(corpus):
        for i in range(1, window_size + 1):
            left_idx = idx - i
            right_idx = idx + i

            if left_idx >= 0:
                left_word_id = corpus[left_idx]
                co_matrix[word_id, left_word_id] += 1

            if right_idx < corpus_size:
                right_word_id = corpus[right_idx]
                co_matrix[word_id, right_word_id] += 1

    return co_matrix

In [123]:
#%%timeit -n 1 
com1 = create_co_matrix(corpus, vocab_size, 1)
#print(com1)

[[0 1 0 0 0 0 0 0 0 0 2 1 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 2 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 2 0 2 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 2 0 1 0 1 0 0 0 0 0 0 0 0 2 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 1 0 1 0 1 0 0 0 0 0 0 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 

In [72]:
%lprun -T create_co_matrix.log -f create_co_matrix create_co_matrix(corpus, vocab_size, 1)
print(open('create_co_matrix.log', 'r').read())


*** Profile printout saved to text file 'create_co_matrix.log'. 
Timer unit: 1e-06 s

Total time: 26.9073 s
File: <ipython-input-70-571efcd7c8c3>
Function: create_co_matrix at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def create_co_matrix(corpus, vocab_size, window_size=1):
     2                                               '''Generate co-occurreance matrix for the corpus.
     3                                           
     4                                               :param corpus:
     5                                               :param vocab_size:The number of unique words in the corpus. 
     6                                               :param window_size: 
     7                                                   The number of words either left or right of the word to count co-occurreances, which is (context_ize / 2)
     8                                               :return: co-occurrenc

## numpy vectorized way

In [95]:
def debug(corpus, index, stride, flag=False):
    if not flag:
        return
    
    n = len(corpus)
    print("word is {} and context is {}".format(
        id_to_word[corpus[index]],
        [ id_to_word[i] for i in corpus[max(0, (index-stride)) : min((index+stride) +1, n)]]
    ))

### Attempt 01

In [74]:
def create_cooccurrence_matrix_01(sequence, vocabrary_size, context_size=3):
    """
    Args: 
        sequence: word index sequence of the original corpus text
        vocabrary_size: number of words in the vocabrary (same with co-occurrence vector size)
        context_size: context (N-gram size N) within to check co-occurrences.
    Returns:
        co_occurrence matrix
    """
    n = sequence_size = len(sequence)
    co_occurrence_matrix = np.zeros((vocabrary_size, vocabrary_size), dtype=np.int32)

    stride = int((context_size - 1)/2 )
    assert(n > stride), "sequence_size {} is less than/equal to stride {}".format(
        n, stride
    )

    # Handle position=slice(0 : (stride-1) +1),       co-occurrences=slice(max(0, position-stride): min((position+stride),n-1) +1)
    # Handle position=slice((n-1-stride) : (n-1) +1), co-occurrences=slice(max(0, position-stride): min((position+stride),n-1) +1)
    indices = [*range(0, (stride-1) +1), *range((n-1)-stride +1, (n-1) +1)]
    #print(indices)
    
    for position in indices:
        debug(sequence, position, stride, False)
        co_occurrence_matrix[
            sequence[position],                                                # position to the word
            sequence[max(0, position-stride) : min((position+stride),n-1) +1]  # indices to co-occurence words 
        ] += 1

    
    # Handle position=slice(stride, ((sequence_size-1) - stride) +1)
    for position in range(stride, (sequence_size-1) - stride + 1):        
        co_occurrence_matrix[
            sequence[position],                                    # position to the word
            sequence[(position-stride) : (position + stride + 1)]  # indices to co-occurence words 
        ] += 1
    
    np.fill_diagonal(co_occurrence_matrix, 0)
    
    return co_occurrence_matrix

In [75]:
#%%timeit -n 1 
#com2 = create_cooccurrence_matrix_01(corpus, vocab_size, 3)
#print(com2)
#assert (com1==com2).all()

In [76]:
#%lprun -T create_cooccurrence_matrix_01.log -f create_cooccurrence_matrix_01 create_cooccurrence_matrix_01(corpus, vocab_size, 3)
#print(open('create_cooccurrence_matrix_01.log', 'r').read())

### Attempt 03

In [15]:
def create_cooccurrence_matrix_03(corpus, vector_size, context_size=3):
    """
    Create a co-occurrence matrix for the copus with two paths.
        1. Handle co-occurrences to the right of the word in the context.
        2. Handle co-occurrences to the left  of the word in the context.
    Args: 
        copus: Original text as word indices
        vector_size: Co-occurrence vector size (including the word itself)
        context_size: Context (N-gram) size N
        
    """
    corpus_size = len(corpus)
    co_ccurrence_matrix = np.zeros((vector_size, vector_size), dtype=np.int32)

    stride = int((context_size - 1)/2 )

    # Handle right stride
    for index in range(0, (corpus_size-1) - stride +1):
        debug(corpus, index, stride)
        co_ccurrence_matrix[
            corpus[index],                       # index to the word
            corpus[index+1 : (index+stride) +1]  # indices to right co-occurance words excluding word itself
        ] += 1
        
    # Handle left stride
    for index in range(stride, (corpus_size-1) +1):
        debug(corpus, index, stride) 
        co_ccurrence_matrix[
            corpus[index],                      # index to the word
            corpus[(index-stride) : index]      # indices to left co-occurance words excluding word itself
        ] += 1

    return co_ccurrence_matrix

In [16]:
#%%timeit -n 1 
com3 = create_cooccurrence_matrix_03(corpus, vocab_size, 3)
#print(com3)
assert (com1==com3).all()

In [17]:
%lprun -T create_cooccurrence_matrix_03.log -f create_cooccurrence_matrix_03 create_cooccurrence_matrix_03(corpus, vocab_size, 3)
print(open('create_cooccurrence_matrix_03.log', 'r').read())


*** Profile printout saved to text file 'create_cooccurrence_matrix_03.log'. 
Timer unit: 1e-06 s

Total time: 0.016627 s
File: <ipython-input-15-2561e2b057b6>
Function: create_cooccurrence_matrix_03 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def create_cooccurrence_matrix_03(corpus, vector_size, context_size=3):
     2                                               """
     3                                               Create a co-occurrence matrix for the copus with two paths.
     4                                                   1. Handle co-occurrences to the right of the word in the context.
     5                                                   2. Handle co-occurrences to the left  of the word in the context.
     6                                               Args: 
     7                                                   copus: Original text as word indices
     8                            

In [18]:
def create_cooccurrence_matrix(sequence, vocabrary_size, context_size=3):
    """
    Args: 
        sequence: word index sequence of the original corpus text
        vocabrary_size: number of words in the vocabrary (same with co-occurrence vector size)
        context_size: context (N-gram size N) within to check co-occurrences.
    Returns:
        co_occurrence matrix
    """
    n = sequence_size = len(sequence)
    co_occurrence_matrix = np.zeros((vocabrary_size, vocabrary_size), dtype=np.int32)

    stride = int((context_size - 1)/2 )
    assert(n > stride), "sequence_size {} is less than/equal to stride {}".format(
        n, stride
    )
    for position in range(0, n):        
        co_occurrence_matrix[
            sequence[position],                                                # position  to the word
            sequence[max(0, position-stride) : min((position+stride),n-1) +1]  # positions to co-occurence words 
        ] += 1

    np.fill_diagonal(co_occurrence_matrix, 0)
    
    return co_occurrence_matrix

In [19]:
#%%timeit -n 1 
com5 = create_cooccurrence_matrix(corpus, vocab_size, 3)
print(com5)
assert (com1==com5).all()

[[0 1 0 ... 0 0 0]
 [1 0 2 ... 0 0 0]
 [0 2 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


## Attemp 04

In [20]:
import numpy as np
def xslice(x, slices):
    """Extract slices from array-like
    Args:
        x: array-like
        slices: slice or tuple of slice objects
    """
    if isinstance(slices, tuple):
        if isinstance(x, np.ndarray):
            return np.concatenate([x[_slice] for _slice in slices])
        else:
            return sum((x[s] if isinstance(s, slice) else [x[s]] for s in slices), [])        
    elif isinstance(slices, slice):
        return x[slices]
    else:
        return [x[slices]]

In [27]:
def create_cooccurrence_matrix(sequence, vocabrary_size, context_size=3):
    """
    Args: 
        sequence: word index sequence of the original corpus text
        vocabrary_size: number of words in the vocabrary (same with co-occurrence vector size)
        context_size: context (N-gram size N) within to check co-occurrences.
    Returns:
        co_occurrence matrix
    """
    n = sequence_size = len(sequence)
    co_occurrence_matrix = np.zeros((vocabrary_size, vocabrary_size), dtype=np.int32)

    stride = int((context_size - 1)/2 )
    assert(n > stride), "sequence_size {} is less than/equal to stride {}".format(
        n, stride
    )

    # --------------------------------------------------------------------------------
    # Bug: This does not work as not considering. the duplicates.
    # e.g. stride=2
    # |W|W|W|W|W| If co-occurences are all same word W at the position, need +4 for W
    # |X|X|W|X|X| If co-occurances are all same word X, need +4 for X
    # |X|X|W|Y|Y| If co-occurances X x 2, Y x 2, then need +2 for X and Y respectively.
    # ...
    # Should go through 
    # --------------------------------------------------------------------------------
    """
    for position in range(0, n):        
        co_occurrence_matrix[
            sequence[position],   # position  to the word 
            # positions to co-occurence words, the word itself can co-occurr.
            xslice(
                sequence,
                np.s_[max(0, position-stride): position, position+1 : min((position+stride),n-1) +1]
            )  
        ] += 1
    """
    for position in range(0, n):        
        context = co_occurrence_matrix[
            sequence[position],   # position  to the word 
            # positions to co-occurence words, the word itself can co-occurr.
            xslice(
                sequence,
                np.s_[max(0, position-stride): position, position+1 : min((position+stride),n-1) +1]
            )  
        ]
    

    np.fill_diagonal(co_occurrence_matrix, 0)
    return co_occurrence_matrix

In [115]:
import numpy as np
a = np.arange(15, 50, 3).astype(np.int32)
print(a)

# %%timeit -n 10000 -> 7.17 µs ± 1.17 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
slices = np.r_[1:4, 1:3] 
print(slices)
np.add(a[slices], [1])
a

[15 18 21 24 27 30 33 36 39 42 45 48]
[1 2 3 1 2]


array([15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48], dtype=int32)

## Compare for debug

In [96]:
def create_cooccurrence_matrix(sequence, vocabrary_size, context_size=3):
    """
    Args: 
        sequence: word index sequence of the original corpus text
        vocabrary_size: number of words in the vocabrary (same with co-occurrence vector size)
        context_size: context (N-gram size N) within to check co-occurrences.
    Returns:
        co_occurrence matrix
    """
    n = sequence_size = len(sequence)
    co_occurrence_matrix = np.zeros((vocabrary_size, vocabrary_size), dtype=np.int32)
    test_matrix = np.zeros((vocabrary_size, vocabrary_size), dtype=np.int32)

    stride = int((context_size - 1)/2 )
    assert(n > stride), "sequence_size {} is less than/equal to stride {}".format(
        n, stride
    )


    # Handle initial right
    for index in range(0, stride):
        debug(corpus, index, stride)
        test_matrix[
            corpus[index],                       # index to the word
            corpus[index+1 : (index+stride) +1]  # indices to right co-occurance words excluding word itself
        ] += 1
        
    for position in range(0, stride):        
        co_occurrence_matrix[
            sequence[position],   # position  to the word 
            # positions to co-occurence words, the word itself can co-occurr.
            xslice(
                sequence,
                np.s_[max(0, position-stride): position, position+1 : min((position+stride),n-1) +1]
            )  
        ] += 1

    # Handle right stride
    for index in range(stride, (n-1) - stride +1):
        test_matrix[
            sequence[index],                       # index to the word
            sequence[index+1 : (index+stride) +1]  # indices to right co-occurance words excluding word itself
        ] += 1

        test_matrix[
            sequence[index],                      # index to the word
            sequence[(index-stride) : index]      # indices to left co-occurance words excluding word itself
        ] += 1
        
        co_occurrence_matrix[
            sequence[index],   # position  to the word 
            sequence[max(0, index-stride) : min((index+stride),n-1) +1]  # positions to co-occurence words 
        ] 
        np.fill_diagonal(co_occurrence_matrix, 0)
            
        if(np.array_equal(test_matrix, co_occurrence_matrix)) is not True:
            print("index is {} test index {} matrix {}\n".format(
                index, 
                [
                    sequence[index],                       # index to the word
                    sequence[(index-stride) : (index+stride) +1]  # indices to right co-occurance words excluding word itself
                ],
                test_matrix[
                    [
                        sequence[index],                       # index to the word
                        sequence[(index-stride) : (index+stride) +1]  # indices to right co-occurance words excluding word itself
                    ]
                ]
            ))
            print("co_occurrence_matrix index is {} matrix is {}\n".format(
                [
                    sequence[index],   # position  to the word 
                    sequence[max(0, index-stride) : min((index+stride),n-1) +1]  # positions to co-occurence words 
                ],
                co_occurrence_matrix[          
                    [
                        sequence[index],   # position  to the word 
                        sequence[max(0, index-stride) : min((index+stride),n-1) +1]  # positions to co-occurence words 
                        # positions to co-occurence words, the word itself can co-occurr.
#                        xslice(
#                            sequence,
#                            np.s_[max(0, index-stride): index, index+1 : min((index+stride),n-1) +1]
#                        )  
                    ]
                ]
            ))
            debug(sequence, index, stride, True) 
            print("diff \n{}".format(test_matrix - co_occurrence_matrix))
#            print("co_occurrence_matrix \n{}".format(co_occurrence_matrix))

            assert False

    # Handle left stride
    for index in range((n-1) - stride, (n-1) +1):
        debug(sequence, index, stride) 
        test_matrix[
            sequence[index],                      # index to the word
            sequence[(index-stride) : index]      # indices to left co-occurance words excluding word itself
        ] += 1
    
    for position in range((n-1) - stride +1, n):        
        co_occurrence_matrix[
            sequence[index],   # position  to the word 
            sequence[max(0, index-stride) : min((index+stride),n-1) +1]  # positions to co-occurence words 
        ] += 1


    np.fill_diagonal(co_occurrence_matrix, 0)
    return co_occurrence_matrix

In [ ]:
#%%timeit -n 1 
com4 = create_cooccurrence_matrix(corpus, vocab_size, 3)
#print(com4)
assert np.array_equal(com1, com4)
print(com1-com4)

<ipython-input-96-fd31b4199c87>:68: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  test_matrix[
<ipython-input-96-fd31b4199c87>:80: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  co_occurrence_matrix[


index is 198 test index [26, array([35, 26, 26])] matrix [1 1 1]

co_occurrence_matrix index is [26, array([35, 26, 26])] matrix is [1 0 0]

word is <unk> and context is ['a', '<unk>', '<unk>']


In [23]:
%lprun -T create_cooccurrence_matrix.log -f create_cooccurrence_matrix create_cooccurrence_matrix(corpus, vocab_size, 3)
print(open('create_cooccurrence_matrix.log', 'r').read())


*** Profile printout saved to text file 'create_cooccurrence_matrix.log'. 
Timer unit: 1e-06 s

Total time: 0.009004 s
File: <ipython-input-21-c1947eddb03c>
Function: create_cooccurrence_matrix at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def create_cooccurrence_matrix(sequence, vocabrary_size, context_size=3):
     2                                               """
     3                                               Args: 
     4                                                   sequence: word index sequence of the original corpus text
     5                                                   vocabrary_size: number of words in the vocabrary (same with co-occurrence vector size)
     6                                                   context_size: context (N-gram size N) within to check co-occurrences.
     7                                               Returns:
     8                                     

In [24]:
X = com1 - com4